# **Fintech Performance Analysis | Part 1**

- Datasets available on [**Kaggle**](https://www.kaggle.com/datasets/mustafakeser4/bigquery-fintech-dataset)

# Data Frames

## Data Load

# Data Cleaning

In [3]:
# Merge df_customer and df_loan on 'customer_id'
df_merged = pd.merge(df_customer, df_loan, on='customer_id', how='inner')

# Merge df_loan_with_region with df_merged on 'loan_id'
df_merged = pd.merge(df_merged, df_loan_with_region, on='loan_id', how='inner')
df_merged['emp_length'].nunique()

11

In [4]:
# Merge df_customer and df_loan on 'customer_id'
df_merged = pd.merge(df_customer, df_loan, on='customer_id', how='inner')

# Merge df_loan_with_region with df_merged on 'loan_id'
df_merged = pd.merge(df_merged, df_loan_with_region, on='loan_id', how='inner')

# Data Cleaning and Transformation
df_merged['emp_title'] = df_merged['emp_title'].fillna("Unknown")
df_merged['description'] = df_merged['description'].fillna("No description")
df_merged['emp_length'] = df_merged['emp_length'].fillna("Unknown")

# Drop unnecessary columns
df_merged = df_merged.drop(columns=['annual_inc_joint', 'issue_d', 'notes', 'addr_state', 'loan_amount_y'])

# Rename loan_amount_x to 'loan_amount'
df_merged = df_merged.rename(columns={'loan_amount_x': 'loan_amount'})

# Capitalize the first letter of string values for all columns except 'state'
df_merged = df_merged.apply(
    lambda col: col.str.capitalize() if col.name != 'state' and col.dtype == 'object' else col
)

# Round 'annual_inc' to two decimal places
df_merged['annual_inc'] = df_merged['annual_inc'].round(2)

# Extract the month from 'issue_date' and drop the original column
df_merged['issue_month'] = df_merged['issue_date'].str.split().str[0]
df_merged = df_merged.drop('issue_date', axis=1)

# Rename columns for consistency
df_merged = df_merged.rename(columns={'pymnt_plan': 'payment_plan', 'Tot_cur_bal': 'tot_cur_bal'})

# Reorder columns: move 'issue_month' before 'issue_year'
cols = df_merged.columns.tolist()
new_order = [col for col in cols if col != 'issue_month']  # Remove issue_month temporarily
new_order.insert(new_order.index('issue_year'), 'issue_month')  # Insert issue_month before issue_year

# Position 'region' right after 'state' and 'zip_code' right before 'state'
cols.remove('region')  # Temporarily remove 'region'
cols.insert(cols.index('state') + 1, 'region')  # Insert 'region' after 'state'
cols.remove('zip_code')  # Temporarily remove 'zip_code'
cols.insert(cols.index('state'), 'zip_code')  # Insert 'zip_code' before 'state'
cols.remove('issue_month')  # Temporarily remove 'issue_month'
cols.insert(cols.index('issue_year'), 'issue_month')  # Insert 'payment_plan' before 'state'


# Apply the new column order
df_merged = df_merged[cols]

# Convert 'issue_year' to integer
df_merged['issue_year'] = df_merged['issue_year'].astype(int)

# Replace values in the 'type' column
df_merged['type'] = df_merged['type'].replace('Direct_pay', 'Direct pay')

In [5]:
# Unable column limitation views
pd.set_option('display.max_columns', None)

In [6]:
df_merged.head()

,customer_id,emp_title,emp_length,home_ownership,annual_inc,verification_status,avg_cur_bal,tot_cur_bal,loan_id,loan_status,loan_amount,zip_code,state,region,funded_amount,term,int_rate,installment,grade,issue_month,issue_year,payment_plan,type,purpose,description
0,B'\xa0\xe6\x1a\xc8h\xbd/(t]\x13[\xf5\x0fk\xd9\...,Unknown,Unknown,Rent,25000.0,Verified,6864.0,34322.0,47207,Fully paid,6025.0,010xx,MA,Northeast,6025.0,36 months,0.1599,211.80000,C,April,2017,False,Individual,Credit_card,Credit card refinancing
1,B'c\xe3s\xf0yy7\xba\xda\x16jf\xec\xbe\xc0my\xd...,Unknown,Unknown,Mortgage,50000.0,Not verified,3418.0,23927.0,47190,Current,6025.0,010xx,MA,Northeast,6025.0,36 months,0.1505,209.00999,C,November,2017,False,Individual,Home_improvement,Home improvement
2,B'1\xe3ud\xe4\x13\x1f\x80\x1d\xb1\xa5\x14x\xac...,Care giver,Unknown,Own,46800.0,Verified,48173.0,240865.0,116454,Fully paid,12000.0,010xx,MA,Northeast,12000.0,36 months,0.0649,367.73999,A,December,2014,False,Individual,Debt_consolidation,Debt consolidation
3,B'bb\xf1y\x00\xaf\x96\x85\x16\xaf\xb4z\x1c\xa0...,Unknown,Unknown,Own,65000.0,Verified,2587.0,46567.0,149201,Current,15000.0,010xx,MA,Northeast,15000.0,36 months,0.0980,482.60999,B,January,2016,False,Individual,Credit_card,Credit card refinancing
4,B'\xaazf\x84\x93\xa0w\xb5g\xed0\xd0\xd4\xd4\xc...,Unknown,Unknown,Own,49200.0,Source verified,738.0,5903.0,84884,Current,10000.0,010xx,MA,Northeast,10000.0,36 months,0.0721,309.73999,A,September,2018,False,Individual,Credit_card,Credit card refinancing


In [7]:
# From dataframe to csv
df_merged.to_csv("df_merged.csv", index=False)


In [8]:
from google.colab import files
files.download("df_merged.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## df_loan_count_by_year cleaning

In [9]:
# Change year type from float to int
# Define new clean Dataframe

df_loan_count_by_year['issue_year'] = df_loan_count_by_year['issue_year'].astype(int)
df_loan_count_by_year_clean = df_loan_count_by_year
df_loan_count_by_year_clean.head()

,issue_year,loan_count
0,2017,44435
1,2019,51737
2,2013,13460
3,2018,49333
4,2014,23453


## df_state_region cleaning


In [10]:
# Drop the row with index 30 that display generic and unsuful data
# Reset the index to adjust it
# Define new clean Dataframe

df_state_region = df_state_region.drop(index=30)
df_state_region.reset_index(drop=True, inplace=True)
df_state_region_cleaned = df_state_region
df_state_region_cleaned.head()

,state,subregion,region
0,AK,Pacific,West
1,CA,Pacific,West
2,HI,Pacific,West
3,OR,Pacific,West
4,WA,Pacific,West


## df_loan_purposes cleaning

In [11]:
# df was already clean in initial file
df_loan_purposes

,purpose
0,other
1,car
2,medical
3,debt_consolidation
4,vacation
5,house
6,renewable_energy
7,credit_card
8,small_business
9,moving
